In [1]:
import numpy as np
import tensorflow as tf
tf.keras.backend.clear_session()
from tensorflow import keras
from tensorflow.keras import layers

# 08-TF-AutoGraph
tf.function的一个很酷的新功能是AutoGraph，它允许使用自然的Python语法编写图形代码。

# 一 tf.function装饰器
当使用tf.function注释函数时，可以像调用任何其他函数一样调用它。 它将被编译成图，这意味着可以获得更快执行，更好地在GPU或TPU上运行或导出到SavedModel。

In [2]:
@tf.function
def simple_nn_layer(x, y):
    return tf.nn.relu(tf.matmul(x, y))


x = tf.random.uniform((3, 3))
y = tf.random.uniform((3, 3))

simple_nn_layer(x, y)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0.16079356, 0.21452281, 0.20754547],
       [0.64166474, 1.0237459 , 1.1091696 ],
       [0.60946167, 0.7407483 , 0.91928065]], dtype=float32)>

如果我们检查注释的结果，我们可以看到它是一个特殊的可调用函数，它处理与TensorFlow运行时的所有交互。

In [3]:
simple_nn_layer

如果代码使用多个函数，则无需对它们进行全部注释 - 从带注释函数调用的任何函数也将以图形模式运行。

In [4]:
def linear_layer(x):
    return 2 * x + 1

@tf.function
def deep_net(x):
    return tf.nn.relu(linear_layer(x))

In [5]:
deep_net

In [7]:
linear_layer

<function __main__.linear_layer(x)>

# 二 使用python控制流程
在tf.function中使用依赖于数据的控制流时，可以使用Python控制流语句，AutoGraph会将它们转换为适当的TensorFlow操作。 例如，如果语句依赖于Tensor，则语句将转换为tf.cond（）。

In [8]:
@tf.function
def square_if_positive(x):
  if x > 0:
    x = x * x
  else:
    x = 0
  return x

In [9]:
# 注意这里输入为 tf.constant
print('square_if_positive(2) = {}'.format(square_if_positive(tf.constant(2))))
print('square_if_positive(-2) = {}'.format(square_if_positive(tf.constant(-2))))

square_if_positive(2) = 4
square_if_positive(-2) = 0


AutoGraph支持常见的Python语句，例如while，if，break，continue和return，支持嵌套。 这意味着可以在while和if语句的条件下使用Tensor表达式，或者在for循环中迭代Tensor。

In [10]:
def sum_even(items):
  s = 0
  for c in items:
    if c % 2 > 0:
      continue
    s += c
  return s

In [11]:
sum_even(tf.constant([10, 12, 15, 20]))

<tf.Tensor: shape=(), dtype=int32, numpy=42>

AutoGraph还为高级用户提供了低级API。 例如，我们可以使用它来查看生成的代码。

In [16]:
print(tf.autograph.to_code(sum_even, experimental_optional_features=None))

def tf__sum_even(items):
    with ag__.FunctionScope('sum_even', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()
        s = 0

        def get_state_2():
            return (s,)

        def set_state_2(vars_):
            nonlocal s
            (s,) = vars_

        def loop_body(itr):
            nonlocal s
            c = itr
            continue_ = False

            def get_state():
                return (continue_,)

            def set_state(vars_):
                nonlocal continue_
                (continue_,) = vars_

            def if_body():
                nonlocal continue_
                continue_ = True

            def else_body():
                nonlocal continue_
                pass
            ag__.if_stmt(((ag__.ld(c) % 2) > 0), if_body, else_body, get_state, set_state, ('continue_',), 1)

           

In [17]:
@tf.function
def fizzbuzz(n):
    msg = tf.constant('')
    for i in tf.range(n):
        if tf.equal(i % 3, 0):
            msg += 'Fizz'
        elif tf.equal(i % 5, 0):
            msg += 'Buzz'
        else:
            msg += tf.as_string(i)
        msg += '\n'
    return msg

In [20]:
print(fizzbuzz(tf.constant(15)).numpy().decode())

Fizz
1
2
Fizz
4
Buzz
Fizz
7
8
Fizz
Buzz
11
Fizz
13
14



# 三 Keras和AutoGraph
也可以将tf.function与对象方法一起使用。 例如，可以通过注释模型的调用函数来装饰自定义Keras模型。

In [21]:
class CustomModel(tf.keras.models.Model):

    @tf.function
    def call(self, input_data):
        if tf.reduce_mean(input_data) > 0:
            return input_data
        else:
            return input_data // 2


model = CustomModel()

In [22]:
model(tf.constant([-2, -4]))

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([-1, -2], dtype=int32)>

就像在eager模式下一样，你可以使用带有副作用的操作，比如通常在tf.function中的tf.assign或tf.print，它会插入必要的控件依赖项以确保它们按顺序执行。

In [24]:
v = tf.Variable(5)

@tf.function
def find_next_odd():
    v.assign(v + 1)
    if tf.equal(v % 2, 0):
        v.assign(v + 1)

find_next_odd()

v

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=7>

# 四 用AutoGraph训练一个简单模型

In [25]:
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

def mnist_dataset():
    (x, y), _ = tf.keras.datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x, y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    return ds

train_dataset = mnist_dataset()
model = tf.keras.Sequential((
    tf.keras.layers.Reshape(target_shape=(28 * 28,), input_shape=(28, 28)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10)))
model.build()
optimizer = tf.keras.optimizers.Adam()
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [26]:
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(y, logits)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    compute_accuracy(y, logits)
    return loss


@tf.function
def train(model, optimizer):
    train_ds = mnist_dataset()
    step = 0
    loss = 0.0
    accuracy = 0.0
    for x, y in train_ds:
        step += 1
        loss = train_one_step(model, optimizer, x, y)
        if tf.equal(step % 10, 0):
            tf.print('Step', step, ': loss', loss, '; accuracy', compute_accuracy.result())
    return step, loss, accuracy

step, loss, accuracy = train(model, optimizer)
print('Final step', step, ': loss', loss, '; accuracy', compute_accuracy.result())

Step 10 : loss 1.87191153 ; accuracy 0.291
Step 20 : loss 1.12330246 ; accuracy 0.4915
Step 30 : loss 0.79729563 ; accuracy 0.586
Step 40 : loss 0.664484918 ; accuracy 0.64925
Step 50 : loss 0.623919427 ; accuracy 0.6912
Step 60 : loss 0.357091486 ; accuracy 0.716166675
Step 70 : loss 0.372655898 ; accuracy 0.741
Step 80 : loss 0.480072021 ; accuracy 0.7585
Step 90 : loss 0.482334226 ; accuracy 0.772666693
Step 100 : loss 0.298391968 ; accuracy 0.7852
Step 110 : loss 0.251136392 ; accuracy 0.796909094
Step 120 : loss 0.220183715 ; accuracy 0.806666672
Step 130 : loss 0.267776251 ; accuracy 0.814384639
Step 140 : loss 0.242873058 ; accuracy 0.821714282
Step 150 : loss 0.304112345 ; accuracy 0.827466667
Step 160 : loss 0.317876846 ; accuracy 0.83325
Step 170 : loss 0.283043 ; accuracy 0.838117659
Step 180 : loss 0.508346081 ; accuracy 0.841666639
Step 190 : loss 0.260991722 ; accuracy 0.845947385
Step 200 : loss 0.162355036 ; accuracy 0.8507
Final step tf.Tensor(200, shape=(), dtype=int3

# 五 关于批处理的说明
在实际应用中，批处理对性能至关重要。 转换为AutoGraph的最佳代码是在批处理级别决定控制流的代码。 如果在单个示例级别做出决策，请尝试使用批处理API来维护性能。

In [27]:
def square_if_positive(x):
    return [i ** 2 if i > 0 else i for i in x]

square_if_positive(range(-5, 5))

[-5, -4, -3, -2, -1, 0, 1, 4, 9, 16]

In [28]:
# 在tensorflow中上面的代码应该改成下面所示
@tf.function
def square_if_positive_naive(x):
    result = tf.TensorArray(tf.int32, size=x.shape[0])
    for i in tf.range(x.shape[0]):
        if x[i] > 0:
            result = result.write(i, x[i] ** 2)
        else:
            result = result.write(i, x[i])
    return result.stack()


square_if_positive_naive(tf.range(-5, 5))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16], dtype=int32)>

In [29]:
# 也可以这么写
def square_if_positive_vectorized(x):
    return tf.where(x > 0, x ** 2, x)

square_if_positive_vectorized(tf.range(-5, 5))

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  4,  9, 16], dtype=int32)>